In [ ]:
# !apt-get install -y fonts-nanum
# !fc-cache -fv
# !rm ~/.cache/matplotlib -rf

## EDA

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/DLthon/train (2).csv")
df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='NanumBarunGothic')

plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='class', order=df['class'].value_counts().index)
plt.title('클래스별 데이터 개수 분포')
plt.xlabel('범죄 유형')
plt.ylabel('데이터 수')
plt.xticks(rotation=45)
plt.show()

# 수치로 확인
print(df['class'].value_counts())

근데 만약 우리가 만드는 합성 데이터를 추가한다면 (일반 데이터) -> 불균형 문제가 발생할 수 있음

학습을 위해서는 충분한 데이터가 필요할것 같은데 합성 데이터를 몇개 만들면 좋을지 그리고 어떠한 방식으로 만들어야할지, 불균형 데이터를 어떻게 처리해야할지 등과 같은 부분등을 정해야할것 같음


In [ ]:
# 1. 글자 수 계산
df['char_len'] = df['conversation'].str.len()

# 2. 단어 수 계산 (띄어쓰기 기준)
df['word_cnt'] = df['conversation'].apply(lambda x: len(x.split()))

# 시각화: 글자 수 분포
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(df['char_len'], bins=30, kde=True, color='blue')
plt.title('전체 대화 글자 수 분포')

plt.subplot(1, 2, 2)
sns.boxplot(x='class', y='char_len', data=df)
plt.title('클래스별 글자 수 분포 (Boxplot)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

전체 대화의 글자는 150~250자 사이 가 가장 많음
그러나 몇개는 800자가 넘어가는 매우 긴 대화도 보임

모델링 할때 300-400 정도의 max_seq_len 고민해봐야할듯

In [ ]:
stats = df.groupby('class')['char_len'].agg(['mean', 'median', 'std', 'min', 'max'])
print("--- 클래스별 글자 수 통계 요약 ---")
print(stats)

In [ ]:
# 1. 600자 초과 데이터만 추출
long_conv_df = df[df['char_len'] > 600]

print(f"600자 초과 데이터 개수: {len(long_conv_df)}개")
print("-" * 30)

# 2. 클래스별로 몇 개씩 있는지 확인
print("클래스별 분포:")
print(long_conv_df['class'].value_counts())
print("-" * 30)

# 3. 실제 대화 내용 확인 (상위 5개)
for i, row in long_conv_df.head(5).iterrows():
    print(f"[Index: {row['idx']} | Class: {row['class']} | Length: {row['char_len']}]")
    print(row['conversation'])
    print("-" * 50)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 문장 부호 카운트 함수
def count_punctuation(text):
    return {
        'question_mark': text.count('?'),
        'exclamation_mark': text.count('!')
    }

# 2. 분석 적용
punc_counts = df['conversation'].apply(lambda x: pd.Series(count_punctuation(x)))
df_punc = pd.concat([df['class'], punc_counts], axis=1)

# 3. 클래스별 평균치 확인
punc_stats = df_punc.groupby('class').mean()
print("--- 클래스별 문장 부호 평균 사용 횟수 ---")
print(punc_stats)

# 4. 시각화
punc_stats.plot(kind='bar', figsize=(12, 6))
plt.title('범죄 유형별 문장 부호 사용 패턴')
plt.ylabel('평균 사용 횟수')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 확인할 간투어 리스트 정의
filler_words = ['아', '음', '어', '저기', '그게', '있잖아', '막']

# 2. 간투어 개수 계산 함수
def count_fillers(text):
    counts = {word: text.count(word) for word in filler_words}
    counts['total_fillers'] = sum(counts.values())
    return pd.Series(counts)

# 3. 데이터프레임에 적용
df_fillers = df['conversation'].apply(count_fillers)
df_analysis = pd.concat([df['class'], df_fillers], axis=1)

# 4. 클래스별 평균 간투어 사용 횟수 요약
filler_stats = df_analysis.groupby('class')['total_fillers'].mean().reset_index()

print("--- 클래스별 평균 간투어 사용 횟수 ---")
print(filler_stats)

# 5. 시각화
plt.figure(figsize=(10, 6))
sns.barplot(data=filler_stats, x='class', y='total_fillers')
plt.title('클래스별 간투어(Filler Words) 평균 사용 빈도')
plt.ylabel('평균 사용 횟수')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 클래스별 각 단어의 평균 발생 횟수 히트맵
word_specific_stats = df_analysis.groupby('class')[filler_words].mean()

plt.figure(figsize=(12, 6))
sns.heatmap(word_specific_stats, annot=True, cmap='YlGnBu')
plt.title('클래스별 간투어 상세 분포 (평균)')
plt.show()

In [ ]:
# 'total_fillers' 컬럼은 df_analysis에 있습니다.
# 간투어가 많은 순서대로 상위 10개를 뽑아봅니다.
# df_analysis에서 'total_fillers'를 기준으로 상위 10개를 선택합니다.
# 이 때, 원본 df의 인덱스와 일치하므로 loc를 사용하여 df에서 conversation을 가져옵니다.
top_filler_analysis = df_analysis.nlargest(10, 'total_fillers')
top_filler_samples = df.loc[top_filler_analysis.index, ['class', 'conversation']].copy()
top_filler_samples['total_fillers'] = top_filler_analysis['total_fillers']

for i, row in top_filler_samples.iterrows():
    print(f"[{row['class']}] 간투어 개수: {row['total_fillers']}")
    print(f"내용: {row['conversation']}")
    print("-" * 50)

In [ ]:
df_t = pd.read_csv("/content/drive/MyDrive/DLthon/test (1).csv")
df_t

### 베이스라인 선정

### 데이터 전처리

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/DLthon/train (2).csv")
df_test = pd.read_csv("/content/drive/MyDrive/DLthon/test (1).csv")
data_normal = pd.read_csv("/content/drive/MyDrive/DLthon/normal_conversation.csv")

In [ ]:
data_normal

In [ ]:
data.head()

In [ ]:
df_test.head()

In [ ]:
df_normal =  pd.read_csv("/content/drive/MyDrive/DLthon/normal_conversation.csv")
df_normal.head()

데이터 전처리 (문장부호 정제 + 화자분리를 SEP token추가)

In [ ]:
# import pandas as pd
# import re
# import os

# # 1. 일반 대화 데이터 로드
# df_normal = pd.read_csv("/content/drive/MyDrive/DLthon/normal_conversation.csv")

# # 2. 전처리 함수 (부호 정제 + [SEP] 삽입)
# def final_preprocessing(text):
#     if not isinstance(text, str): return text
#     # 문장 부호 정제
#     text = re.sub(r'!{2,}', '!', text)
#     text = re.sub(r'\?{2,}', '?', text)
#     text = re.sub(r'\.{2,}', '.', text)
#     # \n -> [SEP] 치환
#     lines = [line.strip() for line in text.split('\n') if line.strip()]
#     return " [SEP] ".join(lines)

# # 3. 각 데이터프레임 전처리 적용
# # Train (기존 범죄 대화)
# data['conversation'] = data['conversation'].apply(final_preprocessing)

# # Normal (일반 대화)
# col_normal = 'conversation' if 'conversation' in df_normal.columns else 'text'
# df_normal[col_normal] = df_normal[col_normal].apply(final_preprocessing)
# # 만약 일반 대화에 class 컬럼이 없다면 추가 (라벨링을 위해)
# if 'class' not in df_normal.columns:
#     df_normal['class'] = '일반 대화'

# # Test (테스트 대화)
# col_test = 'text' if 'text' in df_test.columns else 'conversation'
# df_test[col_test] = df_test[col_test].apply(final_preprocessing)

# # 4. 학습 데이터 통합 (범죄 + 일반)
# data_combined = pd.concat([data, df_normal], ignore_index=True)

# # 5. 저장 경로 설정
# save_path = "/content/drive/MyDrive/DLthon/processed_data"
# if not os.path.exists(save_path):
#     os.makedirs(save_path)

# # 6. 최종 파일 저장
# data_combined.to_csv(f"{save_path}/train_combined_sep.csv", index=False, encoding='utf-8-sig')
# df_test.to_csv(f"{save_path}/test_cleaned_sep.csv", index=False, encoding='utf-8-sig')

# print(f"✅ 학습 데이터 통합 완료: {len(data_combined)}개 샘플")
# print(f"✅ 테스트 데이터 전처리 완료: {len(df_test)}개 샘플")
# print(f"📂 저장 경로: {save_path}")

데이터 전처리 (문장 부호만)

In [ ]:
# import pandas as pd
# import re
# import os

# # 1. 일반 대화 데이터 로드
# df_normal = pd.read_csv("/content/drive/MyDrive/DLthon/normal_conversation.csv")

# # 2. 전처리 함수 (부호 정제 + \n을 공백으로 치환)
# def simplify_no_sep(text):
#     if not isinstance(text, str): return text
#     # 문장 부호 정제 (!, ?, . 하나만 남기기)
#     text = re.sub(r'!{2,}', '!', text)
#     text = re.sub(r'\?{2,}', '?', text)
#     text = re.sub(r'\.{2,}', '.', text)
#     # \n을 [SEP] 대신 '공백'으로 치환하여 문장을 하나로 잇기
#     processed_text = " ".join([line.strip() for line in text.split('\n') if line.strip()])
#     return processed_text

# # 3. 각 데이터프레임 복사 및 전처리 적용
# # Train (기존 범죄 대화)
# df_train_no_sep = data.copy()
# df_train_no_sep['conversation'] = df_train_no_sep['conversation'].apply(simplify_no_sep)

# # Normal (일반 대화)
# df_normal_no_sep = df_normal.copy()
# col_normal = 'conversation' if 'conversation' in df_normal_no_sep.columns else 'text'
# df_normal_no_sep[col_normal] = df_normal_no_sep[col_normal].apply(simplify_no_sep)
# if 'class' not in df_normal_no_sep.columns:
#     df_normal_no_sep['class'] = '일반 대화'

# # Test (테스트 대화)
# df_test_no_sep = df_test.copy()
# col_test = 'text' if 'text' in df_test_no_sep.columns else 'conversation'
# df_test_no_sep[col_test] = df_test_no_sep[col_test].apply(simplify_no_sep)

# # 4. 학습 데이터 통합 (범죄 + 일반)
# train_combined_no_sep = pd.concat([df_train_no_sep, df_normal_no_sep], ignore_index=True)

# # 5. 저장 경로 설정 및 파일 저장
# save_path = "/content/drive/MyDrive/DLthon/processed_data"
# if not os.path.exists(save_path):
#     os.makedirs(save_path)

# train_combined_no_sep.to_csv(f"{save_path}/train_combined_no_sep.csv", index=False, encoding='utf-8-sig')
# df_test_no_sep.to_csv(f"{save_path}/test_no_sep.csv", index=False, encoding='utf-8-sig')

# print(f"✅ [No-SEP] 통합 학습 데이터 저장 완료: {len(train_combined_no_sep)}개 샘플")
# print(f"✅ [No-SEP] 테스트 데이터 저장 완료: {len(df_test_no_sep)}개 샘플")

### 토큰화

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/DLthon/processed_data/train_combined_sep.csv")
df_test = pd.read_csv("/content/drive/MyDrive/DLthon/processed_data/test_cleaned_sep.csv")

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# 1. 레이블 인코더 생성 및 학습
le = LabelEncoder()
df['label'] = le.fit_transform(df['class'])
# 클래스-숫자 매핑 결과 저장 (나중에 추론 시 결과 해석을 위해 필요)
mapping = dict(zip(range(len(le.classes_)), le.classes_))
print("📌 레이블 매핑 결과:", mapping)

# 2. 파이토치 텐서로 변환
# 앞서 만든 input_ids와 attention_masks를 함께 사용합니다.
train_labels = torch.tensor(df['label'].values)
print(f"레이블 텐서 생성 완료: {train_labels.shape}")

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding

# 1. 환경 설정
CHECKPOINT_NAME = "klue/bert-base"
tokenizer_pretrained = CHECKPOINT_NAME

# 2. 데이터 분할 (Stratify 적용)
# 학습 데이터와 검증 데이터를 8:2 비율로 나눕니다.
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label']  # 클래스 비율 유지
)

# 3. 커스텀 데이터셋 클래스 (제공해주신 코드 수정)
class TokenDataset(Dataset):
    def __init__(self, dataframe, tokenizer_pretrained):
        self.data = dataframe
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 컬럼명에 맞춰 수정: 'document' -> 'conversation'
        sentence = self.data.iloc[idx]['conversation']
        label = self.data.iloc[idx]['label']

        tokens = self.tokenizer(
            str(sentence),           # 문장 전달
            return_tensors='pt',
            truncation=True,
            padding='max_length',
            max_length=512,          # 길이를 명시적으로 지정
            add_special_tokens=True
        )

        input_ids = tokens['input_ids'].squeeze(0)
        attention_mask = tokens['attention_mask'].squeeze(0)
        token_type_ids = torch.zeros_like(attention_mask)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label, dtype=torch.long)

# 4. Dataset 인스턴스 생성
train_dataset = TokenDataset(train_df, tokenizer_pretrained)
val_dataset = TokenDataset(val_df, tokenizer_pretrained)

# 5. DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)

print(f"✅ DataLoader 구축 완료 (Stratify 적용)")
print(f"학습 데이터 개수: {len(train_df)} | 검증 데이터 개수: {len(val_df)}")

### 토큰화 확인 코드

In [ ]:
# --- 기존 코드 아래에 추가 ---
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_NAME)

# 1. train_loader에서 첫 번째 배치를 꺼내오기
batch_inputs, batch_labels = next(iter(train_loader))

# 2. 첫 번째 샘플 데이터 추출
# batch_inputs는 {'input_ids': ..., 'attention_mask': ..., 'token_type_ids': ...} 형태입니다.
sample_input_ids = batch_inputs['input_ids'][0]
sample_attention_mask = batch_inputs['attention_mask'][0]
sample_label = batch_labels[0]

# 3. 토큰 ID를 실제 텍스트로 디코딩 (복원)
# skip_special_tokens=False로 설정하여 [CLS], [SEP], [PAD]가 어디에 위치하는지 확인합니다.
decoded_sentence = tokenizer.decode(sample_input_ids, skip_special_tokens=False)

print("\n" + "="*50)
print("🔎 [DataLoader 샘플 데이터 확인]")
print("="*50)
print(f"📌 실제 레이블 (숫자): {sample_label.item()} (클래스: {le.inverse_transform([sample_label.item()])[0]})")
print(f"📌 Input IDs 형태: {sample_input_ids.shape}")
print("-" * 50)
print("📌 토큰화 및 복원된 문장 (앞부분 300자):")
print(decoded_sentence[:300] + "...")
print("-" * 50)
print("📌 실제 토큰 ID 리스트 (앞부분 20개):")
print(sample_input_ids[:20].tolist())
print("-" * 50)
print("📌 Attention Mask (앞부분 20개):")
print(sample_attention_mask[:20].tolist())
print("="*50)

### BERT (이미 학습되어 있는거 fine-tuning)

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertModel


class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        self.dr = nn.Dropout(p=dropout_rate)
        # 5 class 분류
        self.fc = nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [ ]:
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

In [ ]:
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import f1_score

def model_train(model, data_loader, loss_fn, optimizer, device):
    model.train()
    running_loss = 0
    corr = 0
    counts = 0

    # F1 Score 계산을 위해 예측값과 실제값을 저장할 리스트
    all_preds = []
    all_labels = []

    progress_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    for idx, (inputs, labels) in enumerate(progress_bar):
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()

        # 모델 출력 및 로짓(logits) 추출
        output = model(**inputs)
        logits = output.logits if hasattr(output, 'logits') else output

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        _, pred = logits.max(dim=1)

        # 통계치 계산
        corr += pred.eq(labels).sum().item()
        counts += len(labels)
        running_loss += loss.item() * labels.size(0)

        # F1 Score용 데이터 수집
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # 진행 바 업데이트 (현재까지의 F1 포함)
        current_f1 = f1_score(all_labels, all_preds, average='weighted')
        progress_bar.set_description(f"Loss: {running_loss/counts:.4f}, Acc: {corr/counts:.4f}, F1: {current_f1:.4f}")

    final_loss = running_loss / len(data_loader.dataset)
    final_acc = corr / len(data_loader.dataset)
    final_f1 = f1_score(all_labels, all_preds, average='weighted')

    return final_loss, final_acc, final_f1

def model_evaluate(model, data_loader, loss_fn, device):
    model.eval()

    running_loss = 0
    corr = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            output = model(**inputs)
            logits = output.logits if hasattr(output, 'logits') else output

            _, pred = logits.max(dim=1)

            corr += torch.sum(pred.eq(labels)).item()
            running_loss += loss_fn(logits, labels).item() * labels.size(0)

            # F1 Score용 데이터 수집
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    final_loss = running_loss / len(data_loader.dataset)
    final_acc = corr / len(data_loader.dataset)
    final_f1 = f1_score(all_labels, all_preds, average='weighted')

    return final_loss, final_acc, final_f1

In [ ]:
num_epochs = 20
model_name = 'bert-kor-base'
min_loss = np.inf

metric_path = f"{model_name}_metrics.txt"

history = {'train_loss': [], 'train_f1': [], 'val_loss': [], 'val_f1': []}

# 🔹 파일 헤더 (처음 한 번만)
with open(metric_path, 'w') as f:
    f.write("Epoch\tTrain_Loss\tTrain_Acc\tTrain_F1\tVal_Loss\tVal_Acc\tVal_F1\n")

for epoch in range(num_epochs):
    # Training
    train_loss, train_acc, train_f1 = model_train(
        bert, train_loader, loss_fn, optimizer, device
    )

    # Evaluation
    val_loss, val_acc, val_f1 = model_evaluate(
        bert, val_loader, loss_fn, device
    )

    # Checkpoint
    if val_loss < min_loss:
        print(f'✨ [INFO] val_loss improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')

    # 콘솔 출력
    print(f'Epoch [{epoch+1:02d}/{num_epochs}]')
    print(f'TRAIN | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}')
    print(f'VALID | Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}')
    print('-' * 50)

    # 🔹 TXT 파일에 metric 저장
    with open(metric_path, 'a') as f:
        f.write(
            f"{epoch+1}\t"
            f"{train_loss:.4f}\t{train_acc:.4f}\t{train_f1:.4f}\t"
            f"{val_loss:.4f}\t{val_acc:.4f}\t{val_f1:.4f}\n"
        )


In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 최적의 모델 불러오기
bert.load_state_dict(torch.load(f'{model_name}.pth'))
bert.to(device)
bert.eval()

all_preds = []
all_labels = []

# 2. 추론 진행
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = bert(input_ids, attention_mask, token_type_ids)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# 3. 리포트 출력
target_names = ['협박', '갈취', '직장 내 괴롭힘', '기타 괴롭힘', '일반 대화']
print("\n" + "="*60)
print(" [최종 모델 클래스별 성능 리포트] ")
print("="*60)
print(classification_report(all_labels, all_preds, target_names=target_names))